# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use pandas to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!

In [1]:
# Import some libs

import pandas
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Import amazon_baby.csv data to pandas dataframe
products_df = pandas.read_csv('/content/drive/MyDrive/FUNIX Progress/MLP303x_1.1-A_EN/data/amazon_baby.csv')

In [3]:
products_df.head(5)

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


### 1. Preprocessing the data
Before using the review data, we need some preprocessing
<br>
First removew punctuation from words
<br>
For example: **good!** --> **good**, **resturant.** --> **resturant**

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(string.punctuation)

products_df = products_df.fillna({'review':''})  # fill in N/A's in the review column
products_df['review_clean'] = products_df['review'].apply(remove_punctuation)

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.

In [5]:
products_df = products_df[products_df['rating'] != 3]

Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the rating column. In Pandas, you would use apply():

In [6]:
products_df['sentiment'] = products_df['rating'].apply(lambda rating : +1 if rating > 3 else -1)

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set.
<br>
We set the seed to make sure we get the same results.

In [7]:
np.random.seed(1)

train_data, test_data = train_test_split(products_df, test_size=0.2)

### 2. Feature representation
 We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors.
<br>
You can checkout the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from scikit documentation.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

### 3. Train logistic regression model
Learn a logistic regression classifier using the training data. We are using scikit-learn, you should create an instance of the [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (**train_matrix**) as features and the column **sentiment** of **train_data** as the target. Use the default values for other parameters.

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(train_matrix, train_data['sentiment'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Check train accuracy.
<br>
You should get the train accuracy: 0.9474291796913067

In [10]:
print('Train accuracy: {}'.format(clf.score(train_matrix, train_data['sentiment'])))

Train accuracy: 0.9474816530610715


Check test accuracy. 
You should get the test accuracy: 0.9345746754220263

In [11]:
print ('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.9344247548799136


**Quiz question**: How many weights are >= 0?
<br>
**Your answer**:
<br>
`sum(1 for c in clf.coef_[0] if c >= 0)`

In [12]:
sum(1 for c in clf.coef_[0] if c >= 0)

40188

### 4. Making prediction

First, will calculate the scores for 6 samples from the **test_data**.
<br>
Recall from the lecture that: $$\mbox{score}_i = \mathbf{w}^\intercal h(\mathbf{x}_i)$$

In [13]:
sample_test_data = test_data[105:110]
print(sample_test_data['review_clean'])
print(sample_test_data['rating'])
print(sample_test_data['sentiment'])

76151     It came to me with marks on it.I recently went...
164181    How did we survive before discovering Kidzikoo...
108065    My baby loves this jumper! He'll stay playing ...
176539    Looked at this set at Walmart for $50 and coul...
56851     Product was not working when received.  Batter...
Name: review_clean, dtype: object
76151     2
164181    5
108065    5
176539    5
56851     1
Name: rating, dtype: int64
76151    -1
164181    1
108065    1
176539    1
56851    -1
Name: sentiment, dtype: int64


In [14]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = clf.decision_function(sample_test_matrix)
print(scores)

[-6.32190725  6.11927398  4.37471512  2.00441806 -5.66336975]


These scores can be used to make class predictions as follows:
<br>
$$\hat{y}_i = \begin{cases} +1 & \text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) > 0\\ -1 &\text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) \leq 0 \end{cases}$$

We can use the predict() function from scikit-learn.

In [15]:
print(clf.predict(sample_test_matrix))

[-1  1  1  1 -1]


 Recall from the lectures that we can also calculate the probability predictions from the scores using:
 <br>
 $$P(y_i = +1 | \mathbf{x}_i, \mathbf{w}) = \dfrac{1}{1+\exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i))}}$$

We can use the predict_proba() function from scikit-learn.

In [16]:
print(clf.predict_proba(sample_test_matrix))
# NOTE: P(yi=+1|xi,w) represented in the second column, the first column presents the P(yi=-1|xi,w)

[[0.99820671 0.00179329]
 [0.00219522 0.99780478]
 [0.01243515 0.98756485]
 [0.11873983 0.88126017]
 [0.9965412  0.0034588 ]]


We can calculate the accuracy:
<br>
$$\mbox{accuracy} = \dfrac{\mbox{# correctly classified examples}}{\mbox{# total examples}}$$
<br>
We can use the score() function from scikit-learn.

In [17]:
# Accuracy of 6 samples above :DD
print(clf.score(sample_test_matrix, sample_test_data['sentiment']))

1.0


In [18]:
# Accuracy for the hole test_data
print('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.9344247548799136
